In [1]:
import wfdb 
import numpy as np 
import csv
import pickle 
import pandas as pd 
import matplotlib.pyplot as plt

# testing on 1 patient record 

In [2]:
FEATURES = ['Gestation',  
'Age', 
'Parity', 
'Abortions', 
'Weight', 
'Hypertension', 
'Diabetes', 
'Placental_position', 
'Bleeding_first_trimester', 
'Bleeding_second_trimester', 
'Funneling', 
'Smoker']
print(len(FEATURES))

12


In [3]:
file_name = 'records.pkl'
records = []
with open(file_name, 'rb') as f:  
    records = pickle.load(f)
    record = records[0]
    signals, fields = wfdb.rdsamp(record.record_name,pb_dir='tpehgdb/tpehgdb')

In [4]:
fields['comments']

['Comments:',
 'RecID 1007',
 'Gestation 35',
 'Rectime 31.3',
 'Age 30',
 'Parity 0',
 'Abortions 0',
 'Weight 58',
 'Hypertension no',
 'Diabetes no',
 'Placental_position front',
 'Bleeding_first_trimester no',
 'Bleeding_second_trimester no',
 'Funneling negative',
 'Smoker no']

Create patient's list of values, excluding `Comments` and `Record ID` and `Record time`

In [5]:
patient_record_dict = {}
patient_record_values_list = []
for field in fields['comments'][2:]:
#     print(field)
    field_key_val_list = field.split()
    patient_record_values_list.append(field_key_val_list[1])
del patient_record_values_list[1]
print(len(patient_record_values_list))
print((patient_record_values_list))

12
['35', '30', '0', '0', '58', 'no', 'no', 'front', 'no', 'no', 'negative', 'no']


initialize all patient value array

In [6]:
full_arr = np.empty((0,0))
full_arr

array([], shape=(0, 0), dtype=float64)

append current patients value to all patient value array

In [7]:
# list -> ndarray
arr = np.asarray(patient_record_values_list)
full_arr = np.append(full_arr, arr)
full_arr = np.append(full_arr, arr)
full_arr.shape

(24,)

reshaping the full array 

In [8]:
FFFF = (2,12)
full_arr = full_arr.reshape(FFFF)
full_arr.shape

(2, 12)

turn into pandas data frame

In [9]:
rec = pd.DataFrame(full_arr)
rec

,0,1,2,3,4,5,6,7,8,9,10,11
0,35,30,0,0,58,no,no,front,no,no,negative,no
1,35,30,0,0,58,no,no,front,no,no,negative,no


change column names

In [10]:
rec.columns = FEATURES

In [11]:
rec

,Gestation,Age,Parity,Abortions,Weight,Hypertension,Diabetes,Placental_position,Bleeding_first_trimester,Bleeding_second_trimester,Funneling,Smoker
0,35,30,0,0,58,no,no,front,no,no,negative,no
1,35,30,0,0,58,no,no,front,no,no,negative,no


In [50]:
all_df = pd.read_csv('all_patient_records.csv')
all_df = all_df.drop(all_df.columns[0],axis=1)
all_df.head()

,Gestation,Age,Parity,Abortions,Weight,Hypertension,Diabetes,Placental_position,Bleeding_first_trimester,Bleeding_second_trimester,Funneling,Smoker
0,35.0,30,0,0,58,no,no,front,no,no,negative,no
1,38.6,None,None,None,63,None,None,end,yes,no,negative,None
2,38.6,None,None,None,70,None,None,end,yes,no,negative,None
3,37.1,27,0,1,100,no,no,front,no,no,negative,no
4,38.6,28,0,2,72,no,no,front,no,no,negative,no


In [39]:
import signalFeatures as sf
fvlFile = "tpehgdb_features__filter_0.08_Hz-4.0_Hz.fvl"
canceled_index = []
fourFeatures = sf.rdFVL(fvlFile, canceled_index, 1)
fourFeaturesNames = ['Root Mean Square', 'Median Frequency', 'Peak Frequency', 'Sample Entropy']
ff_df = pd.DataFrame(fourFeatures, columns=fourFeaturesNames)
ff_df.head()

,Root Mean Square,Median Frequency,Peak Frequency,Sample Entropy
0,27.9148,0.3109,0.3159,0.502
1,54.2390,0.1426,0.1126,0.243
2,26.1735,0.1598,0.1049,0.396
3,7.5545,0.1750,0.1522,0.732
4,12.7331,0.1816,0.1816,0.817


In [51]:
new_df = all_df.join(ff_df)
new_df.head()

,Gestation,Age,Parity,Abortions,Weight,Hypertension,Diabetes,Placental_position,Bleeding_first_trimester,Bleeding_second_trimester,Funneling,Smoker,Root Mean Square,Median Frequency,Peak Frequency,Sample Entropy
0,35.0,30,0,0,58,no,no,front,no,no,negative,no,27.9148,0.3109,0.3159,0.502
1,38.6,None,None,None,63,None,None,end,yes,no,negative,None,54.2390,0.1426,0.1126,0.243
2,38.6,None,None,None,70,None,None,end,yes,no,negative,None,26.1735,0.1598,0.1049,0.396
3,37.1,27,0,1,100,no,no,front,no,no,negative,no,7.5545,0.1750,0.1522,0.732
4,38.6,28,0,2,72,no,no,front,no,no,negative,no,12.7331,0.1816,0.1816,0.817


In [52]:
new_df.to_csv('all_records_and_signals.csv', index=False)
all_df = pd.read_csv('all_records_and_signals.csv')
all_df.head()

,Gestation,Age,Parity,Abortions,Weight,Hypertension,Diabetes,Placental_position,Bleeding_first_trimester,Bleeding_second_trimester,Funneling,Smoker,Root Mean Square,Median Frequency,Peak Frequency,Sample Entropy
0,35.0,30,0,0,58,no,no,front,no,no,negative,no,27.9148,0.3109,0.3159,0.502
1,38.6,None,None,None,63,None,None,end,yes,no,negative,None,54.2390,0.1426,0.1126,0.243
2,38.6,None,None,None,70,None,None,end,yes,no,negative,None,26.1735,0.1598,0.1049,0.396
3,37.1,27,0,1,100,no,no,front,no,no,negative,no,7.5545,0.1750,0.1522,0.732
4,38.6,28,0,2,72,no,no,front,no,no,negative,no,12.7331,0.1816,0.1816,0.817
